# ⚡ OPTIMIZACIÓN: Reutilizar Resultados de Fases Anteriores

**Este notebook ha sido optimizado para reutilizar resultados existentes:**

- ✅ **Fase 2 (Baseline)**: Carga predicciones desde `../fase 2/outputs/baseline/preds_raw.json`
- ✅ **Fase 3 (MC-Dropout)**: Carga predicciones desde `../fase 3/outputs/mc_dropout/preds_mc_aggregated.json`
- ✅ **Fase 4 (Temperature Scaling)**: Carga temperaturas desde `../fase 4/outputs/temperature_scaling/temperature.json`

**Ventajas**:
- 🚀 Reduce tiempo de ejecución de ~2 horas a ~15 minutos
- 💾 Evita recalcular predicciones costosas (especialmente MC-Dropout con K=5)
- ♻️ Garantiza consistencia con resultados de fases anteriores

**Modo de operación**:
- Si los archivos existen → Los carga y reutiliza
- Si no existen → Ejecuta inferencia completa (fallback)

# Fase 5: Comparación Completa de Métodos de Incertidumbre y Calibración

**Objetivo**: Comparar 6 métodos lado a lado en detección, calibración y risk-coverage.

**Métodos evaluados**:
1. Baseline (sin incertidumbre, sin calibración)
2. Baseline + TS
3. MC-Dropout K=5
4. MC-Dropout K=5 + TS
5. Varianza entre capas (single-pass)
6. Varianza entre capas + TS

**Splits**:
- val_calib: ajustar temperaturas
- val_eval: evaluación final

**Métricas**:
- Detección: mAP@[0.5:0.95], AP50, AP75, por clase
- Calibración: NLL, Brier, ECE, Reliability Diagrams
- Risk-Coverage: curvas y AUC

## 1. Configuración e Imports

In [1]:
# ✅ EJECUTAR PARA RQ1 - Celda 1: Configuración e Imports

import os
import sys
import json
import yaml
import time
import torch
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torchvision
import warnings
warnings.filterwarnings('ignore')

# Configuración
BASE_DIR = Path('..')
DATA_DIR = BASE_DIR / 'data'
OUTPUT_DIR = Path('./outputs/comparison')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

CONFIG = {
    'seed': 42,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'categories': ['person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'traffic light', 'traffic sign'],
    'iou_matching': 0.5,
    'conf_threshold': 0.25,
    'nms_threshold': 0.65,
    'K_mc': 5,
    'n_bins': 10
}

torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
if torch.cuda.is_available():
    torch.cuda.manual_seed(CONFIG['seed'])

with open(OUTPUT_DIR / 'config.yaml', 'w') as f:
    yaml.dump(CONFIG, f)

print(f"Device: {CONFIG['device']}")
print(f"Output: {OUTPUT_DIR}")
print(f"Config guardado")

Device: cuda
Output: outputs/comparison
Config guardado


## 1.1 Cargar Resultados de Fases Anteriores (Optimización)

In [2]:
# ✅ EJECUTAR PARA RQ1 - Celda 2: Cargar resultados de fases anteriores

# Rutas a resultados de fases anteriores
FASE2_BASELINE = BASE_DIR / 'fase 2' / 'outputs' / 'baseline' / 'preds_raw.json'
FASE3_MC_DROPOUT_PARQUET = BASE_DIR / 'fase 3' / 'outputs' / 'mc_dropout' / 'mc_stats_labeled.parquet'
FASE3_MC_DROPOUT_JSON = BASE_DIR / 'fase 3' / 'outputs' / 'mc_dropout' / 'preds_mc_aggregated.json'
FASE4_TEMPERATURE = BASE_DIR / 'fase 4' / 'outputs' / 'temperature_scaling' / 'temperature.json'
FASE4_CALIB_DATA = BASE_DIR / 'fase 4' / 'outputs' / 'temperature_scaling' / 'calib_detections.csv'

# Diccionario para almacenar predicciones cargadas
cached_predictions = {
    'baseline': None,
    'mc_dropout': None,
    'temperatures': None
}

# Cargar Baseline (Fase 2)
if FASE2_BASELINE.exists():
    print(f"✅ Cargando predicciones Baseline desde Fase 2...")
    with open(FASE2_BASELINE, 'r') as f:
        cached_predictions['baseline'] = json.load(f)
    print(f"   → {len(cached_predictions['baseline'])} predicciones cargadas")
else:
    print(f"⚠️  No se encontró {FASE2_BASELINE}, se ejecutará inferencia completa")

# Cargar MC-Dropout (Fase 3) - PRIORIZAR PARQUET CON INCERTIDUMBRE
if FASE3_MC_DROPOUT_PARQUET.exists():
    print(f"✅ Cargando predicciones MC-Dropout desde Fase 3 (con incertidumbre)...")
    mc_df = pd.read_parquet(FASE3_MC_DROPOUT_PARQUET)
    # Convertir a formato JSON similar
    cached_predictions['mc_dropout'] = []
    for _, row in mc_df.iterrows():
        # Convertir bbox de xyxy a xywh para consistencia
        bbox = row['bbox']
        if isinstance(bbox, (list, np.ndarray)) and len(bbox) == 4:
            # Si está en formato xyxy, convertir a xywh
            if bbox[2] > bbox[0] and bbox[3] > bbox[1]:
                bbox_xywh = [bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]]
            else:
                bbox_xywh = bbox
        else:
            bbox_xywh = bbox
            
        cached_predictions['mc_dropout'].append({
            'image_id': int(row['image_id']),
            'category_id': int(row['category_id']) + 1,  # Convertir de 0-indexed a 1-indexed
            'bbox': bbox_xywh,
            'score': float(row['score_mean']),
            'uncertainty': float(row['uncertainty'])  # ¡IMPORTANTE: con incertidumbre!
        })
    print(f"   → {len(cached_predictions['mc_dropout'])} predicciones cargadas (con incertidumbre)")
elif FASE3_MC_DROPOUT_JSON.exists():
    print(f"⚠️  Cargando MC-Dropout desde JSON (SIN incertidumbre)...")
    with open(FASE3_MC_DROPOUT_JSON, 'r') as f:
        cached_predictions['mc_dropout'] = json.load(f)
    print(f"   → {len(cached_predictions['mc_dropout'])} predicciones cargadas")
    print(f"   ⚠️  ADVERTENCIA: Este archivo NO contiene incertidumbre, se calculará como 0.0")
else:
    print(f"⚠️  No se encontró {FASE3_MC_DROPOUT_PARQUET}, se ejecutará inferencia completa")

# Cargar Temperaturas (Fase 4)
if FASE4_TEMPERATURE.exists():
    print(f"✅ Cargando temperaturas optimizadas desde Fase 4...")
    with open(FASE4_TEMPERATURE, 'r') as f:
        cached_predictions['temperatures'] = json.load(f)
    print(f"   → Temperatura baseline: {cached_predictions['temperatures'].get('optimal_temperature', 'N/A')}")
else:
    print(f"⚠️  No se encontró {FASE4_TEMPERATURE}, se calcularán temperaturas")

print(f"\n{'='*60}")
print(f"RESUMEN DE OPTIMIZACIÓN:")
print(f"{'='*60}")
print(f"Baseline disponible:      {'✅ SÍ' if cached_predictions['baseline'] else '❌ NO (se calculará)'}")
print(f"MC-Dropout disponible:    {'✅ SÍ' if cached_predictions['mc_dropout'] else '❌ NO (se calculará)'}")
print(f"Temperaturas disponibles: {'✅ SÍ' if cached_predictions['temperatures'] else '❌ NO (se calcularán)'}")
print(f"{'='*60}\n")

✅ Cargando predicciones Baseline desde Fase 2...
   → 22162 predicciones cargadas
✅ Cargando predicciones MC-Dropout desde Fase 3 (con incertidumbre)...
   → 29914 predicciones cargadas (con incertidumbre)
✅ Cargando temperaturas optimizadas desde Fase 4...
   → Temperatura baseline: N/A

RESUMEN DE OPTIMIZACIÓN:
Baseline disponible:      ✅ SÍ
MC-Dropout disponible:    ✅ SÍ
Temperaturas disponibles: ✅ SÍ



In [3]:
# Funciones para convertir formatos de predicciones desde fases anteriores

def convert_baseline_predictions(baseline_data, image_filename_to_id):
    """
    Convierte predicciones de fase 2 (baseline) al formato esperado.
    baseline_data: lista de dicts con keys: image_id, category_id, bbox, score
    """
    converted = {}
    for pred in baseline_data:
        img_id = pred.get('image_id')
        if img_id not in converted:
            converted[img_id] = []
        
        bbox = pred['bbox']  # [x, y, w, h] en formato COCO
        bbox_xyxy = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        score = pred['score']
        score_clipped = np.clip(score, 1e-7, 1 - 1e-7)
        logit = np.log(score_clipped / (1 - score_clipped))
        
        converted[img_id].append({
            'bbox': bbox_xyxy,
            'score': score_clipped,
            'logit': logit,
            'category_id': pred['category_id'],
            'uncertainty': pred.get('uncertainty', 0.0)  # Baseline no tiene incertidumbre
        })
    
    return converted

def convert_mc_predictions(mc_data, image_filename_to_id):
    """
    Convierte predicciones de fase 3 (MC-Dropout) al formato esperado.
    Maneja tanto bbox en formato [x,y,w,h] como [x1,y1,x2,y2]
    """
    converted = {}
    for pred in mc_data:
        img_id = pred.get('image_id')
        if img_id not in converted:
            converted[img_id] = []
        
        bbox = pred['bbox']  # Podría ser [x, y, w, h] o [x1, y1, x2, y2]
        
        # Detectar formato: si bbox[2] > bbox[0] y bbox[3] > bbox[1], probablemente es xywh
        # Si bbox[2] < bbox[0] o bbox[3] < bbox[1], está mal
        # Asumimos que si w,h > x,y entonces es xyxy
        if len(bbox) == 4:
            # Si parece formato xywh (w,h son razonables)
            if bbox[2] < bbox[0] or bbox[3] < bbox[1]:
                # Ya está en formato xyxy
                bbox_xyxy = bbox
            else:
                # Formato xywh, convertir a xyxy
                bbox_xyxy = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
        else:
            bbox_xyxy = bbox
        
        score = pred['score']
        score_clipped = np.clip(score, 1e-7, 1 - 1e-7)
        logit = np.log(score_clipped / (1 - score_clipped))
        
        converted[img_id].append({
            'bbox': bbox_xyxy,
            'score': score_clipped,
            'logit': logit,
            'category_id': pred['category_id'],
            'uncertainty': pred.get('uncertainty', 0.0)  # ¡IMPORTANTE: preservar incertidumbre!
        })
    
    return converted

print("✅ Funciones de conversión de formato definidas")

✅ Funciones de conversión de formato definidas


## 2. Cargar Modelo y Preparar Funciones

In [4]:
# ✅ EJECUTAR PARA RQ1 - Celda 3: Cargar modelo Grounding DINO

from groundingdino.util.inference import load_model, load_image, predict
from groundingdino.util import box_ops

model_config = '/opt/program/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py'
model_weights = '/opt/program/GroundingDINO/weights/groundingdino_swint_ogc.pth'

model = load_model(model_config, model_weights)
model.to(CONFIG['device'])

TEXT_PROMPT = '. '.join(CONFIG['categories']) + '.'

print(f"Modelo cargado en {CONFIG['device']}")
print(f"Prompt: {TEXT_PROMPT}")

# Guardar referencias de módulos dropout
dropout_modules = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Dropout) and ('class_embed' in name or 'bbox_embed' in name):
        dropout_modules.append(module)

print(f"Módulos dropout en cabeza: {len(dropout_modules)}")

final text_encoder_type: bert-base-uncased
Modelo cargado en cuda
Prompt: person. rider. car. truck. bus. train. motorcycle. bicycle. traffic light. traffic sign.
Módulos dropout en cabeza: 0


In [5]:
# ✅ EJECUTAR PARA RQ1 - Celda 4: Funciones auxiliares

def normalize_label(label):
    synonyms = {'bike': 'bicycle', 'motorbike': 'motorcycle', 'pedestrian': 'person', 
                'stop sign': 'traffic sign', 'red light': 'traffic light'}
    label_lower = label.lower().strip()
    if label_lower in synonyms:
        return synonyms[label_lower]
    for cat in CONFIG['categories']:
        if cat in label_lower:
            return cat
    return label_lower

def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union = area1 + area2 - inter
    return inter / union if union > 0 else 0

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def apply_nms(detections, iou_thresh=0.65):
    if len(detections) == 0:
        return []
    boxes_t = torch.tensor([d['bbox'] for d in detections], dtype=torch.float32)
    scores_t = torch.tensor([d['score'] for d in detections], dtype=torch.float32)
    keep = torchvision.ops.nms(boxes_t, scores_t, iou_thresh)
    return [detections[i] for i in keep.numpy()]

print("Funciones auxiliares definidas")

Funciones auxiliares definidas


## 3. Métodos de Inferencia

In [6]:
def inference_baseline(model, image_path, text_prompt, conf_thresh, device):
    """Método 1: Baseline single-pass sin incertidumbre"""
    model.eval()
    for module in dropout_modules:
        module.eval()
    
    image_source, image = load_image(str(image_path))
    boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
    
    if len(boxes) == 0:
        return []
    
    h, w = image_source.shape[:2]
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
        cat = normalize_label(phrase)
        if cat in CONFIG['categories']:
            score_clipped = np.clip(float(score), 1e-7, 1 - 1e-7)
            logit = np.log(score_clipped / (1 - score_clipped))
            detections.append({
                'bbox': box.tolist(),
                'score': score_clipped,
                'logit': logit,
                'category': cat,
                'uncertainty': 0.0  # Sin incertidumbre
            })
    
    return apply_nms(detections, CONFIG['nms_threshold'])

print("Método 1: Baseline definido")

Método 1: Baseline definido


In [7]:
def inference_mc_dropout(model, image_path, text_prompt, conf_thresh, device, K=5):
    """Método 3: MC-Dropout con K pases"""
    model.eval()
    for module in dropout_modules:
        module.train()
    
    image_source, image = load_image(str(image_path))
    h, w = image_source.shape[:2]
    
    all_detections_k = []
    
    with torch.no_grad():
        for k in range(K):
            boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
            
            if len(boxes) == 0:
                all_detections_k.append([])
                continue
            
            boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
            
            dets_k = []
            for box, score, phrase in zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases):
                cat = normalize_label(phrase)
                if cat in CONFIG['categories']:
                    score_clipped = np.clip(float(score), 1e-7, 1 - 1e-7)
                    dets_k.append({
                        'bbox': box.tolist(),
                        'score': score_clipped,
                        'category': cat
                    })
            all_detections_k.append(dets_k)
    
    # Alinear detecciones entre pases
    if len(all_detections_k) == 0 or all(len(d) == 0 for d in all_detections_k):
        return []
    
    # Usar primer pase como referencia
    ref_dets = all_detections_k[0]
    
    aggregated = []
    for ref_det in ref_dets:
        scores_aligned = [ref_det['score']]
        
        for k in range(1, K):
            best_iou = 0
            best_score = None
            for det_k in all_detections_k[k]:
                if det_k['category'] != ref_det['category']:
                    continue
                iou = compute_iou(ref_det['bbox'], det_k['bbox'])
                if iou > best_iou:
                    best_iou = iou
                    best_score = det_k['score']
            
            if best_iou >= 0.5 and best_score is not None:
                scores_aligned.append(best_score)
        
        mean_score = np.mean(scores_aligned)
        variance = np.var(scores_aligned) if len(scores_aligned) > 1 else 0.0
        
        mean_score_clipped = np.clip(mean_score, 1e-7, 1 - 1e-7)
        logit = np.log(mean_score_clipped / (1 - mean_score_clipped))
        
        aggregated.append({
            'bbox': ref_det['bbox'],
            'score': mean_score_clipped,
            'logit': logit,
            'category': ref_det['category'],
            'uncertainty': variance
        })
    
    return apply_nms(aggregated, CONFIG['nms_threshold'])

print("Método 3: MC-Dropout definido")

Método 3: MC-Dropout definido


In [8]:
# ✅ EJECUTAR PARA RQ1 - Celda 5: Función inference_decoder_variance (MODIFICADA)

def inference_decoder_variance(model, image_path, text_prompt, conf_thresh, device):
    """Método 5: Varianza entre capas del decoder (single-pass)"""
    model.eval()
    for module in dropout_modules:
        module.eval()
    
    image_source, image = load_image(str(image_path))
    h, w = image_source.shape[:2]
    
    # Hook para capturar logits de cada capa del decoder
    layer_logits = []
    
    def hook_fn(module, input, output):
        # Capturar salida de cada capa del decoder
        if isinstance(output, tuple) and len(output) > 0:
            layer_logits.append(output[0].detach() if hasattr(output[0], 'detach') else output[0])
        elif hasattr(output, 'detach'):
            layer_logits.append(output.detach())
    
    # Registrar hooks en capas del decoder
    hooks = []
    for name, module in model.named_modules():
        # Buscar módulos como 'transformer.decoder.layers.0', 'transformer.decoder.layers.1', etc.
        if 'decoder.layers' in name and name.count('.') == 3 and name.split('.')[-1].isdigit():
            hooks.append(module.register_forward_hook(hook_fn))
    
    # Inferencia
    boxes, scores, phrases = predict(model, image, text_prompt, conf_thresh, 0.25, device)
    
    # Remover hooks
    for hook in hooks:
        hook.remove()
    
    if len(boxes) == 0:
        return []
    
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.tensor([w, h, w, h])
    
    detections = []
    for idx, (box, score, phrase) in enumerate(zip(boxes_xyxy.cpu().numpy(), scores.cpu().numpy(), phrases)):
        cat = normalize_label(phrase)
        if cat in CONFIG['categories']:
            score_clipped = np.clip(float(score), 1e-7, 1 - 1e-7)
            logit = np.log(score_clipped / (1 - score_clipped))
            
            # Calcular varianza entre capas si disponible
            uncertainty = 0.0
            layer_uncertainties_list = []
            
            if len(layer_logits) > 0:
                # Los layer_logits son embeddings [900, 1, 256]
                # Necesitamos extraer el embedding de esta detección (query idx)
                # y calcular similitud/score para cada capa
                layer_scores = []
                
                for layer_emb in layer_logits:
                    # layer_emb shape: [num_queries, batch, embed_dim]
                    # Extraer embedding de esta query (detección)
                    if idx < layer_emb.shape[0]:
                        query_emb = layer_emb[idx, 0, :]  # [256]
                        # Calcular score basado en norma del embedding
                        # (queries más confiables tienen normas más altas)
                        emb_norm = torch.norm(query_emb).item()
                        # Normalizar a [0, 1] aproximadamente
                        layer_score = 1.0 / (1.0 + np.exp(-emb_norm / 10.0))
                        layer_scores.append(layer_score)
                
                if len(layer_scores) > 1:
                    uncertainty = np.var(layer_scores)
                    layer_uncertainties_list = layer_scores
            
            detections.append({
                'bbox': box.tolist(),
                'score': score_clipped,
                'logit': logit,
                'category': cat,
                'uncertainty': uncertainty,
                'layer_uncertainties': layer_uncertainties_list,  # ✅ NUEVO: Incertidumbres por capa
                'layer_count': len(layer_uncertainties_list)  # ✅ NUEVO: Número de capas capturadas
            })
    
    return apply_nms(detections, CONFIG['nms_threshold'])

print("Método 5: Decoder variance definido (con layer_uncertainties)")

Método 5: Decoder variance definido (con layer_uncertainties)


## 4. Inferencia en val_calib para Ajustar Temperaturas

### 4.1 Estrategia de Optimización

**Si hay predicciones cacheadas**:
- Baseline → Se cargan de Fase 2
- MC-Dropout → Se cargan de Fase 3
- Solo se calcula Decoder Variance (método nuevo)

**Si NO hay predicciones cacheadas**:
- Se ejecuta inferencia completa para todos los métodos

In [9]:
val_eval_json = DATA_DIR / 'bdd100k_coco/val_eval.json'
image_dir = DATA_DIR / 'bdd100k/bdd100k/bdd100k/images/100k/val'

coco_eval_full = COCO(str(val_eval_json))
img_ids_all = coco_eval_full.getImgIds()

# Split inteligente de val_eval (2000 imágenes):
# - Primeras 500 para calibración (ajustar temperaturas)
# - Restantes 1500 para evaluación final
img_ids_calib = img_ids_all[:500]
img_ids_eval_final = img_ids_all[500:]

print(f"📊 ESTRATEGIA DE SPLITS:")
print(f"  Dataset: val_eval.json (2,000 imágenes)")
print(f"  ├─ Calibración: {len(img_ids_calib)} imágenes (primeras 500)")
print(f"  └─ Evaluación:  {len(img_ids_eval_final)} imágenes (restantes 1,500)")

# Crear COCO object para calibración
coco_calib = COCO(str(val_eval_json))

print(f"\nProcesando {len(img_ids_calib)} imágenes para ajustar temperaturas...")

methods_calib_data = {
    'baseline': [],
    'mc_dropout': [],
    'decoder_variance': []
}

# Contadores para diagnóstico
counters = {
    'baseline_cached': 0,
    'baseline_computed': 0,
    'mc_cached': 0,
    'mc_computed': 0
}

# ============================================================================
# OPTIMIZACIÓN: Usar predicciones cacheadas si están disponibles
# ============================================================================

# Convertir predicciones cacheadas a formato útil
baseline_by_img = {}
mc_by_img = {}

if cached_predictions['baseline']:
    print("\n✅ Usando predicciones Baseline cacheadas de Fase 2")
    baseline_by_img = convert_baseline_predictions(cached_predictions['baseline'], {})
    print(f"   → {len(baseline_by_img)} imágenes indexadas")
    
if cached_predictions['mc_dropout']:
    print("✅ Usando predicciones MC-Dropout cacheadas de Fase 3")
    mc_by_img = convert_mc_predictions(cached_predictions['mc_dropout'], {})
    print(f"   → {len(mc_by_img)} imágenes indexadas")

# Verificar overlap con primeras 500 imágenes de val_eval
calib_500 = set(img_ids_calib)
baseline_overlap = set(baseline_by_img.keys()) & calib_500
mc_overlap = set(mc_by_img.keys()) & calib_500

print(f"\n🔍 OVERLAP CON CALIBRACIÓN (primeras 500 de val_eval):")
print(f"   Baseline cacheado: {len(baseline_overlap)}/500 imágenes ({len(baseline_overlap)/500*100:.1f}%)")
print(f"   MC-Dropout cacheado: {len(mc_overlap)}/500 imágenes ({len(mc_overlap)/500*100:.1f}%)")

if len(baseline_overlap) < 500:
    print(f"   ⚠️  {500 - len(baseline_overlap)} imágenes de baseline se calcularán desde cero")
if len(mc_overlap) < 500:
    print(f"   ⚠️  {500 - len(mc_overlap)} imágenes de MC-Dropout se calcularán desde cero")
    print(f"   ⏱️  Tiempo estimado: ~{(500 - len(mc_overlap)) * 1.8 / 60:.1f} minutos")

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!
📊 ESTRATEGIA DE SPLITS:
  Dataset: val_eval.json (2,000 imágenes)
  ├─ Calibración: 500 imágenes (primeras 500)
  └─ Evaluación:  1500 imágenes (restantes 1,500)
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!

Procesando 500 imágenes para ajustar temperaturas...

✅ Usando predicciones Baseline cacheadas de Fase 2
   → 1988 imágenes indexadas
✅ Usando predicciones MC-Dropout cacheadas de Fase 3
   → 1996 imágenes indexadas

🔍 OVERLAP CON CALIBRACIÓN (primeras 500 de val_eval):
   Baseline cacheado: 497/500 imágenes (99.4%)
   MC-Dropout cacheado: 498/500 imágenes (99.6%)
   ⚠️  3 imágenes de baseline se calcularán desde cero
   ⚠️  2 imágenes de MC-Dropout se calcularán desde cero
   ⏱️  Tiempo estimado: ~0.1 minutos


### 🐛 DEBUG: Probar inference_decoder_variance con 1 imagen

In [10]:
# 🐛 CELDA DE DEBUG - Ejecutar solo UNA imagen para diagnosticar hooks

print("🐛 DEBUG: Probando inference_decoder_variance con una imagen...")

# Tomar la primera imagen de calibración
test_img_id = img_ids_calib[0]
test_img_info = coco_calib.loadImgs(test_img_id)[0]
test_img_path = image_dir / test_img_info['file_name']

print(f"   Imagen de prueba: {test_img_info['file_name']} (ID: {test_img_id})")
print(f"   Ruta: {test_img_path}")
print(f"   Existe: {test_img_path.exists()}")

if test_img_path.exists():
    print(f"\n🔍 Ejecutando inference_decoder_variance...")
    test_preds = inference_decoder_variance(model, test_img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'])
    
    print(f"\n📊 RESULTADOS:")
    print(f"   Detecciones: {len(test_preds)}")
    
    if len(test_preds) > 0:
        print(f"\n   Primera detección:")
        first_pred = test_preds[0]
        for key, value in first_pred.items():
            print(f"      {key}: {value}")
        
        # Verificar layer_uncertainties
        if 'layer_uncertainties' in first_pred:
            layer_unc = first_pred['layer_uncertainties']
            print(f"\n   ✅ layer_uncertainties presente: {len(layer_unc)} valores")
            if len(layer_unc) > 0:
                print(f"      Valores: {layer_unc}")
            else:
                print(f"      ⚠️  VACÍO - Este es el problema que debemos solucionar")
        else:
            print(f"\n   ❌ layer_uncertainties NO está en la salida")
    else:
        print(f"   ⚠️  No se detectaron objetos en esta imagen")
else:
    print(f"   ❌ Imagen no encontrada")

print(f"\n{'='*70}")
"corrido"

🐛 DEBUG: Probando inference_decoder_variance con una imagen...
   Imagen de prueba: c8c97803-657086fb.jpg (ID: 9306)
   Ruta: ../data/bdd100k/bdd100k/bdd100k/images/100k/val/c8c97803-657086fb.jpg
   Existe: True

🔍 Ejecutando inference_decoder_variance...

📊 RESULTADOS:
   Detecciones: 22

   Primera detección:
      bbox: [246.6544952392578, 344.82891845703125, 390.66827392578125, 422.9021301269531]
      score: 0.6032488942146301
      logit: 0.4190207249455299
      category: car
      uncertainty: 0.0012345988001037553
      layer_uncertainties: [0.7025667871442604, 0.7570377492065832, 0.7397240831970635, 0.7811286757848407, 0.8028580751556345, 0.7985424860655854]
      layer_count: 6

   ✅ layer_uncertainties presente: 6 valores
      Valores: [0.7025667871442604, 0.7570377492065832, 0.7397240831970635, 0.7811286757848407, 0.8028580751556345, 0.7985424860655854]



'corrido'

In [11]:
# Procesar imágenes de calibración
for img_id in tqdm(img_ids_calib, desc="Procesando calibración"):
    img_info = coco_calib.loadImgs(img_id)[0]
    img_path = image_dir / img_info['file_name']
    
    if not img_path.exists():
        continue
    
    gt_anns = coco_calib.loadAnns(coco_calib.getAnnIds(imgIds=img_id))
    
    # ========================================================================
    # Método 1: Baseline
    # ========================================================================
    if img_id in baseline_by_img:
        # Usar predicciones cacheadas
        preds_baseline = baseline_by_img[img_id]
        counters['baseline_cached'] += 1
    else:
        # Calcular desde cero
        counters['baseline_computed'] += 1
        preds_baseline_raw = inference_baseline(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'])
        preds_baseline = []
        for pred in preds_baseline_raw:
            cat_id = CONFIG['categories'].index(pred['category']) + 1
            preds_baseline.append({
                'bbox': pred['bbox'],
                'score': pred['score'],
                'logit': pred['logit'],
                'category_id': cat_id,
                'uncertainty': pred['uncertainty']
            })
    
    # Etiquetar como TP/FP
    for pred in preds_baseline:
        is_tp = 0
        cat_id = pred['category_id']
        cat = CONFIG['categories'][cat_id - 1] if 1 <= cat_id <= len(CONFIG['categories']) else ''
        
        for gt in gt_anns:
            if gt['category_id'] != cat_id:
                continue
            gt_box = gt['bbox']
            gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
            if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                is_tp = 1
                break
        
        methods_calib_data['baseline'].append({
            'logit': pred['logit'],
            'score': pred['score'],
            'category': cat,
            'uncertainty': pred['uncertainty'],
            'is_tp': is_tp
        })
    
    # ========================================================================
    # Método 3: MC-Dropout
    # ========================================================================
    if img_id in mc_by_img:
        # Usar predicciones cacheadas
        preds_mc = mc_by_img[img_id]
        counters['mc_cached'] += 1
    else:
        # Calcular desde cero
        counters['mc_computed'] += 1
        preds_mc_raw = inference_mc_dropout(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'], CONFIG['K_mc'])
        preds_mc = []
        for pred in preds_mc_raw:
            cat_id = CONFIG['categories'].index(pred['category']) + 1
            preds_mc.append({
                'bbox': pred['bbox'],
                'score': pred['score'],
                'logit': pred['logit'],
                'category_id': cat_id,
                'uncertainty': pred['uncertainty']
            })
    
    # Etiquetar como TP/FP
    for pred in preds_mc:
        is_tp = 0
        cat_id = pred['category_id']
        cat = CONFIG['categories'][cat_id - 1] if 1 <= cat_id <= len(CONFIG['categories']) else ''
        
        for gt in gt_anns:
            if gt['category_id'] != cat_id:
                continue
            gt_box = gt['bbox']
            gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
            if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                is_tp = 1
                break
        
        methods_calib_data['mc_dropout'].append({
            'logit': pred['logit'],
            'score': pred['score'],
            'category': cat,
            'uncertainty': pred['uncertainty'],
            'is_tp': is_tp
        })
    
    # ========================================================================
    # Método 5: Decoder Variance (siempre se calcula, es nuevo)
    # ========================================================================
    preds_dec = inference_decoder_variance(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'])
    for pred in preds_dec:
        is_tp = 0
        cat_id = CONFIG['categories'].index(pred['category']) + 1
        cat = pred['category']
        
        for gt in gt_anns:
            if gt['category_id'] != cat_id:
                continue
            gt_box = gt['bbox']
            gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
            if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                is_tp = 1
                break
        
        methods_calib_data['decoder_variance'].append({
            'logit': pred['logit'],
            'score': pred['score'],
            'category': cat,
            'uncertainty': pred['uncertainty'],
            'is_tp': is_tp
        })

# Mostrar contadores
print(f"\n📊 ESTADÍSTICAS DE PROCESAMIENTO:")
print(f"   Baseline: {counters['baseline_cached']} cacheadas, {counters['baseline_computed']} calculadas")
print(f"   MC-Dropout: {counters['mc_cached']} cacheadas, {counters['mc_computed']} calculadas")

# Guardar datos de calibración
for method_name, data in methods_calib_data.items():
    df = pd.DataFrame(data)
    df.to_csv(OUTPUT_DIR / f'calib_{method_name}.csv', index=False)
    print(f"\n{method_name}: {len(df)} detecciones, TP={df['is_tp'].sum()}")

print("\n✅ Datos de calibración guardados")

# DIAGNÓSTICO FINAL: Verificar si los CSVs son diferentes
print(f"\n🔍 DIAGNÓSTICO FINAL - Verificando CSVs de calibración:")
df_baseline = pd.read_csv(OUTPUT_DIR / 'calib_baseline.csv')
df_mc = pd.read_csv(OUTPUT_DIR / 'calib_mc_dropout.csv')
print(f"   Baseline: {len(df_baseline)} registros, uncertainty media={df_baseline['uncertainty'].mean():.6f}")
print(f"   MC-Dropout: {len(df_mc)} registros, uncertainty media={df_mc['uncertainty'].mean():.6f}")

# Comparar primeras 5 logits
print(f"\n   Primeras 5 logits de cada método:")
print(f"   Baseline:   {df_baseline['logit'].head().tolist()}")
print(f"   MC-Dropout: {df_mc['logit'].head().tolist()}")

if df_baseline['logit'].head(10).equals(df_mc['logit'].head(10)):
    print(f"   ⚠️  Los primeros 10 logits son idénticos (puede ser coincidencia o problema)")
else:
    print(f"   ✅ Los logits son diferentes")
    
if df_baseline['uncertainty'].equals(df_mc['uncertainty']):
    print(f"   ⚠️  Las incertidumbres son idénticas")
else:
    print(f"   ✅ Las incertidumbres son diferentes")

Procesando calibración: 100%|██████████| 500/500 [02:50<00:00,  2.93it/s]


📊 ESTADÍSTICAS DE PROCESAMIENTO:
   Baseline: 497 cacheadas, 3 calculadas
   MC-Dropout: 498 cacheadas, 2 calculadas

baseline: 5457 detecciones, TP=3558

mc_dropout: 7387 detecciones, TP=51

decoder_variance: 7453 detecciones, TP=4336

✅ Datos de calibración guardados

🔍 DIAGNÓSTICO FINAL - Verificando CSVs de calibración:
   Baseline: 5457 registros, uncertainty media=0.000000
   MC-Dropout: 7387 registros, uncertainty media=0.000089

   Primeras 5 logits de cada método:
   Baseline:   [0.3380101929716149, -0.1017526020559519, 0.4190207249455299, -0.2801209492000665, -0.0834355056962531]
   MC-Dropout: [-0.1081413067481472, -0.2560700530227012, -0.4148847215280034, -0.5178735333913581, -0.7217361723679339]
   ✅ Los logits son diferentes
   ✅ Las incertidumbres son diferentes


## 5. Optimizar Temperaturas

In [12]:
from scipy.optimize import minimize

def nll_loss(T, logits, labels):
    T = max(T, 0.01)
    probs = sigmoid(logits / T)
    probs = np.clip(probs, 1e-7, 1 - 1e-7)
    return -np.mean(labels * np.log(probs) + (1 - labels) * np.log(1 - probs))

# ============================================================================
# OPTIMIZACIÓN: Usar temperaturas de Fase 4 si están disponibles
# ============================================================================

if cached_predictions['temperatures'] and 'optimal_temperature' in cached_predictions['temperatures']:
    print("✅ Usando temperatura optimizada de Fase 4")
    T_baseline = cached_predictions['temperatures']['optimal_temperature']
    
    # Calcular NLL antes y después con esta temperatura
    df_baseline = pd.read_csv(OUTPUT_DIR / 'calib_baseline.csv')
    logits_baseline = df_baseline['logit'].values
    labels_baseline = df_baseline['is_tp'].values
    
    nll_before = nll_loss(1.0, logits_baseline, labels_baseline)
    nll_after = nll_loss(T_baseline, logits_baseline, labels_baseline)
    
    temperatures = {
        'baseline': {
            'T': T_baseline,
            'nll_before': nll_before,
            'nll_after': nll_after,
            'source': 'cached_from_fase4'
        }
    }
    
    print(f"  baseline: T={T_baseline:.4f}, NLL: {nll_before:.4f} → {nll_after:.4f} (cacheada)")
    
else:
    print("⚙️  Calculando temperaturas desde cero...")
    temperatures = {}

# Calcular temperaturas para MC-Dropout y Decoder Variance (siempre, son nuevos)
for method_name in ['mc_dropout', 'decoder_variance']:
    df = pd.read_csv(OUTPUT_DIR / f'calib_{method_name}.csv')
    logits = df['logit'].values
    labels = df['is_tp'].values
    
    nll_before = nll_loss(1.0, logits, labels)
    result = minimize(lambda T: nll_loss(T, logits, labels), x0=1.0, bounds=[(0.01, 10.0)], method='L-BFGS-B')
    T_opt = result.x[0]
    nll_after = result.fun
    
    temperatures[method_name] = {
        'T': T_opt,
        'nll_before': nll_before,
        'nll_after': nll_after,
        'source': 'calculated'
    }
    
    print(f"  {method_name}: T={T_opt:.4f}, NLL: {nll_before:.4f} → {nll_after:.4f}")

# Si no había temperatura cacheada para baseline, calcularla
if 'baseline' not in temperatures:
    print("⚙️  Calculando temperatura para baseline...")
    df = pd.read_csv(OUTPUT_DIR / 'calib_baseline.csv')
    logits = df['logit'].values
    labels = df['is_tp'].values
    
    nll_before = nll_loss(1.0, logits, labels)
    result = minimize(lambda T: nll_loss(T, logits, labels), x0=1.0, bounds=[(0.01, 10.0)], method='L-BFGS-B')
    T_opt = result.x[0]
    nll_after = result.fun
    
    temperatures['baseline'] = {
        'T': T_opt,
        'nll_before': nll_before,
        'nll_after': nll_after,
        'source': 'calculated'
    }
    
    print(f"  baseline: T={T_opt:.4f}, NLL: {nll_before:.4f} → {nll_after:.4f}")

with open(OUTPUT_DIR / 'temperatures.json', 'w') as f:
    json.dump(temperatures, f, indent=2)

print(f"\n✅ Temperaturas guardadas en: {OUTPUT_DIR / 'temperatures.json'}")

⚙️  Calculando temperaturas desde cero...
  mc_dropout: T=0.3192, NLL: 0.5123 → 0.4001
  decoder_variance: T=2.6534, NLL: 0.7061 → 0.6850
⚙️  Calculando temperatura para baseline...
  baseline: T=4.2128, NLL: 0.7107 → 0.6912

✅ Temperaturas guardadas en: outputs/comparison/temperatures.json


## 6. Evaluación en val_eval con COCO API

In [13]:
# ✅ EJECUTAR PARA RQ1 - Celda 6: Ejecutar inferencia en val_eval (1500 imágenes)
# Esta celda ejecuta decoder_variance en TODAS las imágenes de evaluación

# ============================================================================
# EVALUACIÓN: Usar restantes 1500 imágenes de val_eval
# ============================================================================
# Las primeras 500 se usaron para calibración, ahora usamos el resto

print(f"\n{'='*70}")
print(f"EVALUACIÓN EN VAL_EVAL (1,500 imágenes restantes)")
print(f"{'='*70}")

coco_eval = COCO(str(val_eval_json))

print(f"Procesando {len(img_ids_eval_final)} imágenes para evaluación final...")

methods_results = {
    'baseline': [],
    'baseline_ts': [],
    'mc_dropout': [],
    'mc_dropout_ts': [],
    'decoder_variance': [],
    'decoder_variance_ts': []
}

# Cargar temperaturas
with open(OUTPUT_DIR / 'temperatures.json', 'r') as f:
    temps = json.load(f)

print(f"\n📊 Temperaturas a aplicar:")
for method, temp_info in temps.items():
    print(f"   {method}: T={temp_info['T']:.4f}")

# ============================================================================
# OPTIMIZACIÓN: Construir índices de predicciones cacheadas para evaluación
# ============================================================================

baseline_eval_by_img = {}
mc_eval_by_img = {}

if cached_predictions['baseline']:
    print("\n✅ Indexando predicciones Baseline cacheadas para evaluación")
    for pred in cached_predictions['baseline']:
        img_id = pred.get('image_id')
        if img_id in img_ids_eval_final:  # Solo las de evaluación (no calibración)
            if img_id not in baseline_eval_by_img:
                baseline_eval_by_img[img_id] = []
            baseline_eval_by_img[img_id].append(pred)
    print(f"   → {len(baseline_eval_by_img)} imágenes con predicciones cacheadas")

if cached_predictions['mc_dropout']:
    print("✅ Indexando predicciones MC-Dropout cacheadas para evaluación")
    for pred in cached_predictions['mc_dropout']:
        img_id = pred.get('image_id')
        if img_id in img_ids_eval_final:  # Solo las de evaluación (no calibración)
            if img_id not in mc_eval_by_img:
                mc_eval_by_img[img_id] = []
            mc_eval_by_img[img_id].append(pred)
    print(f"   → {len(mc_eval_by_img)} imágenes con predicciones cacheadas")

# Contadores
eval_counters = {
    'baseline_cached': 0,
    'baseline_computed': 0,
    'mc_cached': 0,
    'mc_computed': 0
}

# Procesar imágenes de evaluación
for img_id in tqdm(img_ids_eval_final, desc="Procesando evaluación"):
    img_info = coco_eval.loadImgs(img_id)[0]
    img_path = image_dir / img_info['file_name']
    
    if not img_path.exists():
        continue
    
    gt_anns = coco_eval.loadAnns(coco_eval.getAnnIds(imgIds=img_id))
    
    # ========================================================================
    # Baseline (sin TS y con TS)
    # ========================================================================
    if img_id in baseline_eval_by_img:
        # Usar cacheadas
        eval_counters['baseline_cached'] += 1
        for pred in baseline_eval_by_img[img_id]:
            is_tp = 0
            bbox = pred['bbox']
            bbox_xyxy = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
            
            for gt in gt_anns:
                if gt['category_id'] != pred['category_id']:
                    continue
                gt_box = gt['bbox']
                gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
                if compute_iou(bbox_xyxy, gt_box_xyxy) >= CONFIG['iou_matching']:
                    is_tp = 1
                    break
            
            score = pred['score']
            score_clipped = np.clip(score, 1e-7, 1 - 1e-7)
            logit = np.log(score_clipped / (1 - score_clipped))
            
            methods_results['baseline'].append({
                'image_id': img_id,
                'category_id': pred['category_id'],
                'bbox': pred['bbox'],
                'score': score_clipped,
                'logit': logit,
                'uncertainty': pred.get('uncertainty', 0.0),
                'is_tp': is_tp
            })
            
            # Con TS
            score_ts = sigmoid(logit / temps['baseline']['T'])
            methods_results['baseline_ts'].append({
                'image_id': img_id,
                'category_id': pred['category_id'],
                'bbox': pred['bbox'],
                'score': score_ts,
                'logit': logit,
                'uncertainty': pred.get('uncertainty', 0.0),
                'is_tp': is_tp
            })
    else:
        # Calcular desde cero
        eval_counters['baseline_computed'] += 1
        preds_baseline = inference_baseline(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'])
        for pred in preds_baseline:
            cat_id = CONFIG['categories'].index(pred['category']) + 1
            is_tp = 0
            for gt in gt_anns:
                if gt['category_id'] != cat_id:
                    continue
                gt_box = gt['bbox']
                gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
                if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                    is_tp = 1
                    break
            
            methods_results['baseline'].append({
                'image_id': img_id,
                'category_id': cat_id,
                'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
                'score': pred['score'],
                'logit': pred['logit'],
                'uncertainty': pred['uncertainty'],
                'is_tp': is_tp
            })
            
            score_ts = sigmoid(pred['logit'] / temps['baseline']['T'])
            methods_results['baseline_ts'].append({
                'image_id': img_id,
                'category_id': cat_id,
                'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
                'score': score_ts,
                'logit': pred['logit'],
                'uncertainty': pred['uncertainty'],
                'is_tp': is_tp
            })
    
    # ========================================================================
    # MC-Dropout (sin TS y con TS)
    # ========================================================================
    if img_id in mc_eval_by_img:
        # Usar cacheadas
        eval_counters['mc_cached'] += 1
        for pred in mc_eval_by_img[img_id]:
            is_tp = 0
            bbox = pred['bbox']
            bbox_xyxy = [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]]
            
            for gt in gt_anns:
                if gt['category_id'] != pred['category_id']:
                    continue
                gt_box = gt['bbox']
                gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
                if compute_iou(bbox_xyxy, gt_box_xyxy) >= CONFIG['iou_matching']:
                    is_tp = 1
                    break
            
            score = pred['score']
            score_clipped = np.clip(score, 1e-7, 1 - 1e-7)
            logit = np.log(score_clipped / (1 - score_clipped))
            
            methods_results['mc_dropout'].append({
                'image_id': img_id,
                'category_id': pred['category_id'],
                'bbox': pred['bbox'],
                'score': score_clipped,
                'logit': logit,
                'uncertainty': pred.get('uncertainty', 0.0),
                'is_tp': is_tp
            })
            
            # Con TS
            score_ts = sigmoid(logit / temps['mc_dropout']['T'])
            methods_results['mc_dropout_ts'].append({
                'image_id': img_id,
                'category_id': pred['category_id'],
                'bbox': pred['bbox'],
                'score': score_ts,
                'logit': logit,
                'uncertainty': pred.get('uncertainty', 0.0),
                'is_tp': is_tp
            })
    else:
        # Calcular desde cero
        eval_counters['mc_computed'] += 1
        preds_mc = inference_mc_dropout(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'], CONFIG['K_mc'])
        for pred in preds_mc:
            cat_id = CONFIG['categories'].index(pred['category']) + 1
            is_tp = 0
            for gt in gt_anns:
                if gt['category_id'] != cat_id:
                    continue
                gt_box = gt['bbox']
                gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
                if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                    is_tp = 1
                    break
            
            methods_results['mc_dropout'].append({
                'image_id': img_id,
                'category_id': cat_id,
                'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
                'score': pred['score'],
                'logit': pred['logit'],
                'uncertainty': pred['uncertainty'],
                'is_tp': is_tp
            })
            
            score_ts = sigmoid(pred['logit'] / temps['mc_dropout']['T'])
            methods_results['mc_dropout_ts'].append({
                'image_id': img_id,
                'category_id': cat_id,
                'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
                'score': score_ts,
                'logit': pred['logit'],
                'uncertainty': pred['uncertainty'],
                'is_tp': is_tp
            })
    
    # ========================================================================
    # Decoder variance (siempre se calcula, es nuevo)
    # ========================================================================
    preds_dec = inference_decoder_variance(model, img_path, TEXT_PROMPT, CONFIG['conf_threshold'], CONFIG['device'])
    for pred in preds_dec:
        cat_id = CONFIG['categories'].index(pred['category']) + 1
        is_tp = 0
        for gt in gt_anns:
            if gt['category_id'] != cat_id:
                continue
            gt_box = gt['bbox']
            gt_box_xyxy = [gt_box[0], gt_box[1], gt_box[0] + gt_box[2], gt_box[1] + gt_box[3]]
            if compute_iou(pred['bbox'], gt_box_xyxy) >= CONFIG['iou_matching']:
                is_tp = 1
                break
        
        methods_results['decoder_variance'].append({
            'image_id': img_id,
            'category_id': cat_id,
            'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
            'score': pred['score'],
            'logit': pred['logit'],
            'uncertainty': pred['uncertainty'],
            'layer_uncertainties': pred.get('layer_uncertainties', []),
            'is_tp': is_tp
        })
        
        score_ts = sigmoid(pred['logit'] / temps['decoder_variance']['T'])
        methods_results['decoder_variance_ts'].append({
            'image_id': img_id,
            'category_id': cat_id,
            'bbox': [pred['bbox'][0], pred['bbox'][1], pred['bbox'][2] - pred['bbox'][0], pred['bbox'][3] - pred['bbox'][1]],
            'score': score_ts,
            'logit': pred['logit'],
            'uncertainty': pred['uncertainty'],
            'layer_uncertainties': pred.get('layer_uncertainties', []),
            'is_tp': is_tp
        })

# Estadísticas finales
print(f"\n📊 ESTADÍSTICAS DE EVALUACIÓN:")
print(f"   Baseline: {eval_counters['baseline_cached']} cacheadas, {eval_counters['baseline_computed']} calculadas")
print(f"   MC-Dropout: {eval_counters['mc_cached']} cacheadas, {eval_counters['mc_computed']} calculadas")

# Guardar resultados (CSV y JSON)
for method_name, results in methods_results.items():
    # Guardar CSV
    df = pd.DataFrame(results)
    df.to_csv(OUTPUT_DIR / f'eval_{method_name}.csv', index=False)
    
    # Guardar JSON (formato compatible con RQ1)
    with open(OUTPUT_DIR / f'eval_{method_name}.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n{method_name}: {len(df)} detecciones")

print(f"\n✅ Resultados de evaluación guardados (CSV + JSON)")


EVALUACIÓN EN VAL_EVAL (1,500 imágenes restantes)
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Procesando 1500 imágenes para evaluación final...

📊 Temperaturas a aplicar:
   mc_dropout: T=0.3192
   decoder_variance: T=2.6534
   baseline: T=4.2128

✅ Indexando predicciones Baseline cacheadas para evaluación
   → 1491 imágenes con predicciones cacheadas
✅ Indexando predicciones MC-Dropout cacheadas para evaluación
   → 1498 imágenes con predicciones cacheadas


Procesando evaluación: 100%|██████████| 1500/1500 [08:37<00:00,  2.90it/s]



📊 ESTADÍSTICAS DE EVALUACIÓN:
   Baseline: 1491 cacheadas, 9 calculadas
   MC-Dropout: 1498 cacheadas, 2 calculadas

baseline: 16724 detecciones

baseline_ts: 16724 detecciones

mc_dropout: 22527 detecciones

mc_dropout_ts: 22527 detecciones

decoder_variance: 22793 detecciones

decoder_variance_ts: 22793 detecciones

✅ Resultados de evaluación guardados (CSV + JSON)


## 7. Calcular Métricas de Detección (mAP)

In [12]:
detection_metrics = {}

for method_name in methods_results.keys():
    print(f"\nEvaluando {method_name}...")
    
    # Cargar predicciones en formato COCO
    preds_file = OUTPUT_DIR / f'eval_{method_name}.json'
    
    if os.path.getsize(preds_file) > 0:
        coco_dt = coco_eval.loadRes(str(preds_file))
        coco_eval_obj = COCOeval(coco_eval, coco_dt, 'bbox')
        coco_eval_obj.evaluate()
        coco_eval_obj.accumulate()
        coco_eval_obj.summarize()
        
        detection_metrics[method_name] = {
            'mAP': coco_eval_obj.stats[0],
            'AP50': coco_eval_obj.stats[1],
            'AP75': coco_eval_obj.stats[2],
            'AP_small': coco_eval_obj.stats[3],
            'AP_medium': coco_eval_obj.stats[4],
            'AP_large': coco_eval_obj.stats[5]
        }
        
        # mAP por clase
        per_class_ap = {}
        for cat_id, cat_name in enumerate(CONFIG['categories'], 1):
            coco_eval_obj.params.catIds = [cat_id]
            coco_eval_obj.evaluate()
            coco_eval_obj.accumulate()
            per_class_ap[cat_name] = coco_eval_obj.stats[0]
        
        detection_metrics[method_name]['per_class'] = per_class_ap
    else:
        detection_metrics[method_name] = {'mAP': 0.0, 'AP50': 0.0, 'AP75': 0.0}

with open(OUTPUT_DIR / 'detection_metrics.json', 'w') as f:
    json.dump(detection_metrics, f, indent=2)

print("\nMétricas de detección guardadas")


Evaluando baseline...
Loading and preparing results...
DONE (t=0.15s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.50s).
Accumulating evaluation results...
DONE (t=0.55s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.170
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.279
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.171
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.063
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.182
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.377
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.188
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.284
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.285
 Average Recall     (AR) @[ IoU=0.50:0.95 | a

## 8. Tabla Comparativa de Detección

In [13]:
with open(OUTPUT_DIR / 'detection_metrics.json', 'r') as f:
    det_metrics = json.load(f)

# Crear tabla comparativa
rows = []
for method_name, metrics in det_metrics.items():
    row = {
        'Method': method_name,
        'mAP': metrics.get('mAP', 0.0),
        'AP50': metrics.get('AP50', 0.0),
        'AP75': metrics.get('AP75', 0.0)
    }
    
    # Agregar mAP por clase principal
    if 'per_class' in metrics:
        for cat in ['person', 'car', 'truck', 'traffic_light', 'traffic_sign']:
            cat_key = cat.replace('_', ' ')
            row[f'AP_{cat}'] = metrics['per_class'].get(cat_key, 0.0)
    
    rows.append(row)

df_detection = pd.DataFrame(rows)
df_detection.to_csv(OUTPUT_DIR / 'detection_comparison.csv', index=False)

print("\n" + "="*80)
print("TABLA COMPARATIVA DE DETECCIÓN")
print("="*80)
print(df_detection.to_string(index=False))
print("="*80)


TABLA COMPARATIVA DE DETECCIÓN
             Method      mAP     AP50     AP75  AP_person   AP_car  AP_truck  AP_traffic_light  AP_traffic_sign
           baseline 0.170481 0.278535 0.170542   0.170481 0.170481  0.170481          0.170481         0.170481
        baseline_ts 0.170481 0.278535 0.170542   0.170481 0.170481  0.170481          0.170481         0.170481
         mc_dropout 0.182274 0.302312 0.181113   0.182274 0.182274  0.182274          0.182274         0.182274
      mc_dropout_ts 0.182274 0.302312 0.181113   0.182274 0.182274  0.182274          0.182274         0.182274
   decoder_variance 0.181892 0.302048 0.180095   0.181892 0.181892  0.181892          0.181892         0.181892
decoder_variance_ts 0.181892 0.302048 0.180095   0.181892 0.181892  0.181892          0.181892         0.181892


## 9. Calcular Métricas de Calibración

In [14]:
def compute_calibration_metrics(logits, labels, T=1.0, n_bins=10):
    probs = sigmoid(logits / T)
    probs = np.clip(probs, 1e-7, 1 - 1e-7)
    
    # NLL
    nll = -np.mean(labels * np.log(probs) + (1 - labels) * np.log(1 - probs))
    
    # Brier
    brier = np.mean((probs - labels) ** 2)
    
    # ECE
    bins = np.linspace(0, 1, n_bins + 1)
    digitized = np.digitize(probs, bins) - 1
    
    ece = 0.0
    bin_data = []
    
    for i in range(n_bins):
        mask = digitized == i
        if mask.sum() > 0:
            conf = probs[mask].mean()
            acc = labels[mask].mean()
            gap = abs(conf - acc)
            ece += gap * mask.sum() / len(probs)
            bin_data.append({
                'bin': i,
                'confidence': conf,
                'accuracy': acc,
                'count': mask.sum()
            })
    
    return {'NLL': nll, 'Brier': brier, 'ECE': ece, 'bin_data': bin_data}

calibration_metrics = {}

for method_name in methods_results.keys():
    df = pd.read_csv(OUTPUT_DIR / f'eval_{method_name}.csv')
    logits = df['logit'].values
    labels = df['is_tp'].values
    
    # Sin TS (T=1)
    if '_ts' not in method_name:
        metrics = compute_calibration_metrics(logits, labels, T=1.0, n_bins=CONFIG['n_bins'])
        calibration_metrics[method_name] = metrics
    else:
        # Con TS
        base_method = method_name.replace('_ts', '')
        T = temps[base_method]['T']
        metrics = compute_calibration_metrics(logits, labels, T=T, n_bins=CONFIG['n_bins'])
        calibration_metrics[method_name] = metrics
    
    print(f"{method_name}: NLL={metrics['NLL']:.4f}, Brier={metrics['Brier']:.4f}, ECE={metrics['ECE']:.4f}")

# Guardar
with open(OUTPUT_DIR / 'calibration_metrics.json', 'w') as f:
    # Convertir para JSON serializable
    cal_save = {}
    for k, v in calibration_metrics.items():
        cal_save[k] = {
            'NLL': v['NLL'],
            'Brier': v['Brier'],
            'ECE': v['ECE']
        }
    json.dump(cal_save, f, indent=2)

print("\nMétricas de calibración guardadas")

baseline: NLL=0.7180, Brier=0.2618, ECE=0.2410
baseline_ts: NLL=0.6930, Brier=0.2499, ECE=0.1868
mc_dropout: NLL=0.7069, Brier=0.2561, ECE=0.2034
mc_dropout_ts: NLL=1.0070, Brier=0.3365, ECE=0.3428
decoder_variance: NLL=0.7093, Brier=0.2572, ECE=0.2065
decoder_variance_ts: NLL=0.6863, Brier=0.2466, ECE=0.1409

Métricas de calibración guardadas


## 10. Tabla Comparativa de Calibración

In [15]:
rows_calib = []
for method_name, metrics in calibration_metrics.items():
    rows_calib.append({
        'Method': method_name,
        'NLL': metrics['NLL'],
        'Brier': metrics['Brier'],
        'ECE': metrics['ECE']
    })

df_calibration = pd.DataFrame(rows_calib)
df_calibration.to_csv(OUTPUT_DIR / 'calibration_comparison.csv', index=False)

print("\n" + "="*80)
print("TABLA COMPARATIVA DE CALIBRACIÓN")
print("="*80)
print(df_calibration.to_string(index=False))
print("="*80)
print("\nInterpretación:")
print("  ↓ Menor es mejor para NLL, Brier, ECE")
print("  Si método+TS < método: TS mejoró calibración")


TABLA COMPARATIVA DE CALIBRACIÓN
             Method      NLL    Brier      ECE
           baseline 0.718032 0.261844 0.240970
        baseline_ts 0.693014 0.249935 0.186833
         mc_dropout 0.706870 0.256100 0.203429
      mc_dropout_ts 1.007017 0.336512 0.342814
   decoder_variance 0.709267 0.257221 0.206473
decoder_variance_ts 0.686261 0.246594 0.140935

Interpretación:
  ↓ Menor es mejor para NLL, Brier, ECE
  Si método+TS < método: TS mejoró calibración


## 11. Reliability Diagrams

In [16]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

method_pairs = [
    ('baseline', 'baseline_ts'),
    ('mc_dropout', 'mc_dropout_ts'),
    ('decoder_variance', 'decoder_variance_ts')
]

for idx, (method_before, method_after) in enumerate(method_pairs):
    ax = axes[idx * 2]
    
    # Sin TS
    bin_data = calibration_metrics[method_before]['bin_data']
    if len(bin_data) > 0:
        confidences = [b['confidence'] for b in bin_data]
        accuracies = [b['accuracy'] for b in bin_data]
        counts = [b['count'] for b in bin_data]
        
        ax.bar(range(len(confidences)), accuracies, alpha=0.3, label='Accuracy', color='blue')
        ax.plot(range(len(confidences)), confidences, 'o-', label='Confidence', color='red', markersize=8)
        ax.plot([0, len(confidences)-1], [0, 1], 'k--', alpha=0.3, label='Perfect calibration')
        ax.set_xlabel('Confidence bin')
        ax.set_ylabel('Proportion')
        ax.set_title(f'{method_before}\nECE={calibration_metrics[method_before]["ECE"]:.4f}')
        ax.legend()
        ax.grid(alpha=0.3)
    
    # Con TS
    ax = axes[idx * 2 + 1]
    bin_data = calibration_metrics[method_after]['bin_data']
    if len(bin_data) > 0:
        confidences = [b['confidence'] for b in bin_data]
        accuracies = [b['accuracy'] for b in bin_data]
        
        ax.bar(range(len(confidences)), accuracies, alpha=0.3, label='Accuracy', color='blue')
        ax.plot(range(len(confidences)), confidences, 'o-', label='Confidence', color='red', markersize=8)
        ax.plot([0, len(confidences)-1], [0, 1], 'k--', alpha=0.3, label='Perfect calibration')
        ax.set_xlabel('Confidence bin')
        ax.set_ylabel('Proportion')
        ax.set_title(f'{method_after}\nECE={calibration_metrics[method_after]["ECE"]:.4f}')
        ax.legend()
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'reliability_diagrams.png', dpi=150, bbox_inches='tight')
print(f"Reliability diagrams guardados en: {OUTPUT_DIR / 'reliability_diagrams.png'}")
plt.close()

Reliability diagrams guardados en: outputs/comparison/reliability_diagrams.png


## 12. Risk-Coverage Analysis

## 13. Métricas de Incertidumbre: AUROC TP vs FP

In [17]:
from sklearn.metrics import roc_auc_score, roc_curve

uncertainty_auroc = {}

# Solo métodos con incertidumbre (MC-Dropout y Decoder Variance)
uncertainty_methods = ['mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']

print("="*80)
print("AUROC: ¿La incertidumbre detecta errores (FP)?")
print("="*80)
print("\nObjetivo: Usar incertidumbre para distinguir FP (errores) de TP (aciertos)")
print("Interpretación: AUROC > 0.5 (random), ideal ≥ 0.7")
print("-"*80)

for method_name in uncertainty_methods:
    df = pd.read_csv(OUTPUT_DIR / f'eval_{method_name}.csv')
    
    if len(df) > 0 and 'uncertainty' in df.columns:
        uncertainties = df['uncertainty'].values
        is_tp = df['is_tp'].values
        
        # Verificar que hay TPs y FPs
        if len(np.unique(is_tp)) > 1 and len(uncertainties) > 0:
            # AUROC: predecir FP (error) usando incertidumbre
            # Invertir labels: 1=FP (error), 0=TP (correcto)
            is_fp = 1 - is_tp
            
            try:
                auroc = roc_auc_score(is_fp, uncertainties)
                
                # Estadísticas de incertidumbre
                unc_tp = uncertainties[is_tp == 1]
                unc_fp = uncertainties[is_tp == 0]
                
                mean_unc_tp = unc_tp.mean() if len(unc_tp) > 0 else 0.0
                mean_unc_fp = unc_fp.mean() if len(unc_fp) > 0 else 0.0
                
                uncertainty_auroc[method_name] = {
                    'auroc': auroc,
                    'mean_unc_tp': mean_unc_tp,
                    'mean_unc_fp': mean_unc_fp,
                    'n_tp': int(is_tp.sum()),
                    'n_fp': int((1 - is_tp).sum())
                }
                
                print(f"\n{method_name}:")
                print(f"  AUROC (FP detection): {auroc:.4f}")
                print(f"  Mean uncertainty TP:  {mean_unc_tp:.6f}")
                print(f"  Mean uncertainty FP:  {mean_unc_fp:.6f}")
                print(f"  Ratio (FP/TP):        {mean_unc_fp/mean_unc_tp if mean_unc_tp > 0 else 0:.2f}x")
                print(f"  Samples: {int(is_tp.sum())} TP, {int((1-is_tp).sum())} FP")
                
            except Exception as e:
                print(f"\n{method_name}: Error calculando AUROC - {e}")
        else:
            print(f"\n{method_name}: Datos insuficientes para AUROC")

# Guardar resultados
with open(OUTPUT_DIR / 'uncertainty_auroc.json', 'w') as f:
    json.dump(uncertainty_auroc, f, indent=2)

print("\n" + "="*80)
print(f"Resultados guardados en: {OUTPUT_DIR / 'uncertainty_auroc.json'}")


AUROC: ¿La incertidumbre detecta errores (FP)?

Objetivo: Usar incertidumbre para distinguir FP (errores) de TP (aciertos)
Interpretación: AUROC > 0.5 (random), ideal ≥ 0.7
--------------------------------------------------------------------------------

mc_dropout:
  AUROC (FP detection): 0.6335
  Mean uncertainty TP:  0.000061
  Mean uncertainty FP:  0.000126
  Ratio (FP/TP):        2.07x
  Samples: 13317 TP, 9210 FP

mc_dropout_ts:
  AUROC (FP detection): 0.6335
  Mean uncertainty TP:  0.000061
  Mean uncertainty FP:  0.000126
  Ratio (FP/TP):        2.07x
  Samples: 13317 TP, 9210 FP

decoder_variance:
  AUROC (FP detection): 0.5000
  Mean uncertainty TP:  0.000000
  Mean uncertainty FP:  0.000000
  Ratio (FP/TP):        0.00x
  Samples: 13508 TP, 9285 FP

decoder_variance_ts:
  AUROC (FP detection): 0.5000
  Mean uncertainty TP:  0.000000
  Mean uncertainty FP:  0.000000
  Ratio (FP/TP):        0.00x
  Samples: 13508 TP, 9285 FP

Resultados guardados en: outputs/comparison/uncerta

In [18]:
# Tabla comparativa AUROC
rows_auroc = []
for method_name, metrics in uncertainty_auroc.items():
    rows_auroc.append({
        'Method': method_name,
        'AUROC (FP detection) ↑': metrics['auroc'],
        'Mean Unc. TP': metrics['mean_unc_tp'],
        'Mean Unc. FP': metrics['mean_unc_fp'],
        'Ratio (FP/TP)': metrics['mean_unc_fp'] / metrics['mean_unc_tp'] if metrics['mean_unc_tp'] > 0 else 0
    })

df_auroc = pd.DataFrame(rows_auroc)
df_auroc.to_csv(OUTPUT_DIR / 'uncertainty_auroc_comparison.csv', index=False)

print("\n" + "="*80)
print("TABLA COMPARATIVA: AUROC TP vs FP")
print("="*80)
print(df_auroc.to_string(index=False))
print("="*80)
print("\nInterpretación:")
print("  ↑ Mayor AUROC = mejor detección de errores")
print("  Ratio (FP/TP) > 1 = incertidumbre mayor en errores (deseable)")
print("  AUROC ≥ 0.7 = incertidumbre útil para rechazo selectivo")



TABLA COMPARATIVA: AUROC TP vs FP
             Method  AUROC (FP detection) ↑  Mean Unc. TP  Mean Unc. FP  Ratio (FP/TP)
         mc_dropout                0.633462      0.000061      0.000126       2.070721
      mc_dropout_ts                0.633462      0.000061      0.000126       2.070721
   decoder_variance                0.500000      0.000000      0.000000       0.000000
decoder_variance_ts                0.500000      0.000000      0.000000       0.000000

Interpretación:
  ↑ Mayor AUROC = mejor detección de errores
  Ratio (FP/TP) > 1 = incertidumbre mayor en errores (deseable)
  AUROC ≥ 0.7 = incertidumbre útil para rechazo selectivo


In [19]:
# Visualización: Distribuciones de incertidumbre y ROC curves
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

methods_to_plot = ['mc_dropout', 'decoder_variance']
colors_methods = {'mc_dropout': 'blue', 'decoder_variance': 'green'}

for idx, method_name in enumerate(methods_to_plot):
    # Fila 1: Distribuciones de incertidumbre (TP vs FP)
    ax_dist = axes[0, idx]
    
    df = pd.read_csv(OUTPUT_DIR / f'eval_{method_name}.csv')
    if len(df) > 0 and 'uncertainty' in df.columns:
        unc_tp = df[df['is_tp'] == 1]['uncertainty'].values
        unc_fp = df[df['is_tp'] == 0]['uncertainty'].values
        
        ax_dist.hist(unc_tp, bins=50, alpha=0.6, label=f'TP (n={len(unc_tp)})', color='green', density=True)
        ax_dist.hist(unc_fp, bins=50, alpha=0.6, label=f'FP (n={len(unc_fp)})', color='red', density=True)
        ax_dist.axvline(unc_tp.mean(), color='green', linestyle='--', linewidth=2, label=f'Mean TP: {unc_tp.mean():.4f}')
        ax_dist.axvline(unc_fp.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean FP: {unc_fp.mean():.4f}')
        ax_dist.set_xlabel('Uncertainty', fontsize=11)
        ax_dist.set_ylabel('Density', fontsize=11)
        ax_dist.set_title(f'{method_name.replace("_", " ").title()}\nDistribución de Incertidumbre', fontsize=12, fontweight='bold')
        ax_dist.legend(fontsize=9)
        ax_dist.grid(alpha=0.3)
    
    # Fila 2: ROC curves
    ax_roc = axes[1, idx]
    
    if method_name in uncertainty_auroc:
        is_tp = df['is_tp'].values
        is_fp = 1 - is_tp
        uncertainties = df['uncertainty'].values
        
        fpr, tpr, thresholds = roc_curve(is_fp, uncertainties)
        auroc = uncertainty_auroc[method_name]['auroc']
        
        ax_roc.plot(fpr, tpr, linewidth=2, label=f'AUROC = {auroc:.4f}', color=colors_methods[method_name])
        ax_roc.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random (0.5)')
        ax_roc.set_xlabel('False Positive Rate', fontsize=11)
        ax_roc.set_ylabel('True Positive Rate', fontsize=11)
        ax_roc.set_title(f'{method_name.replace("_", " ").title()}\nROC Curve (FP Detection)', fontsize=12, fontweight='bold')
        ax_roc.legend(fontsize=10)
        ax_roc.grid(alpha=0.3)
        ax_roc.set_xlim([0, 1])
        ax_roc.set_ylim([0, 1])

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'uncertainty_analysis.png', dpi=150, bbox_inches='tight')
print(f"\nVisualización de incertidumbre guardada en: {OUTPUT_DIR / 'uncertainty_analysis.png'}")
plt.close()



Visualización de incertidumbre guardada en: outputs/comparison/uncertainty_analysis.png


## 12. Risk-Coverage Analysis

In [20]:
def compute_risk_coverage(df, uncertainty_col='uncertainty'):
    """Calcula curva risk-coverage"""
    df_sorted = df.sort_values(uncertainty_col, ascending=False).reset_index(drop=True)
    
    coverages = []
    risks = []
    
    for i in range(1, len(df_sorted) + 1):
        coverage = i / len(df_sorted)
        risk = 1 - df_sorted.iloc[:i]['is_tp'].mean()
        coverages.append(coverage)
        risks.append(risk)
    
    # AUC (área bajo la curva)
    auc = np.trapz(risks, coverages)
    
    return coverages, risks, auc

# Calcular risk-coverage para métodos con incertidumbre
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

methods_with_uncertainty = ['mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']
colors = ['blue', 'cyan', 'red', 'orange']

risk_coverage_results = {}

for ax_idx, method_name in enumerate(['mc_dropout', 'decoder_variance']):
    ax = axes[ax_idx]
    
    for variant, color in [(method_name, 'blue'), (f'{method_name}_ts', 'red')]:
        df = pd.read_csv(OUTPUT_DIR / f'eval_{variant}.csv')
        
        if len(df) > 0 and 'uncertainty' in df.columns:
            coverages, risks, auc = compute_risk_coverage(df, 'uncertainty')
            
            label = variant.replace('_', ' ').title()
            ax.plot(coverages, risks, label=f'{label} (AUC={auc:.3f})', color=color, linewidth=2)
            
            risk_coverage_results[variant] = {
                'coverages': coverages,
                'risks': risks,
                'auc': auc
            }
    
    ax.set_xlabel('Coverage', fontsize=12)
    ax.set_ylabel('Risk (1 - Accuracy)', fontsize=12)
    ax.set_title(f'Risk-Coverage: {method_name.replace("_", " ").title()}', fontsize=14)
    ax.legend(fontsize=10)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'risk_coverage_curves.png', dpi=150, bbox_inches='tight')
print(f"Risk-coverage curves guardadas en: {OUTPUT_DIR / 'risk_coverage_curves.png'}")
plt.close()

# Guardar AUC
auc_summary = {k: v['auc'] for k, v in risk_coverage_results.items()}
with open(OUTPUT_DIR / 'risk_coverage_auc.json', 'w') as f:
    json.dump(auc_summary, f, indent=2)

print("\nRisk-Coverage AUC:")
for method, auc in auc_summary.items():
    print(f"  {method}: {auc:.4f}")

Risk-coverage curves guardadas en: outputs/comparison/risk_coverage_curves.png

Risk-Coverage AUC:
  mc_dropout: 0.5245
  mc_dropout_ts: 0.5245
  decoder_variance: 0.4101
  decoder_variance_ts: 0.4101


## 14. Resumen Final y Reporte

In [21]:
print("\n" + "="*80)
print("RESUMEN FINAL - COMPARACIÓN DE MÉTODOS")
print("="*80)

# Cargar todas las métricas
det_metrics = json.load(open(OUTPUT_DIR / 'detection_metrics.json'))
cal_metrics = json.load(open(OUTPUT_DIR / 'calibration_metrics.json'))
temps = json.load(open(OUTPUT_DIR / 'temperatures.json'))
auc_summary = json.load(open(OUTPUT_DIR / 'risk_coverage_auc.json'))
uncertainty_auroc_data = json.load(open(OUTPUT_DIR / 'uncertainty_auroc.json'))

print("\n1. MÉTRICAS DE DETECCIÓN (mAP@[0.5:0.95])")
print("-" * 80)
for method in ['baseline', 'baseline_ts', 'mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']:
    mAP = det_metrics[method].get('mAP', 0.0)
    AP50 = det_metrics[method].get('AP50', 0.0)
    AP75 = det_metrics[method].get('AP75', 0.0)
    print(f"{method:25s}  mAP={mAP:.4f}  AP50={AP50:.4f}  AP75={AP75:.4f}")

print("\n2. MÉTRICAS DE CALIBRACIÓN")
print("-" * 80)
print(f"{'Method':<25s} {'NLL ↓':>10s} {'Brier ↓':>10s} {'ECE ↓':>10s}")
print("-" * 80)
for method in ['baseline', 'baseline_ts', 'mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']:
    nll = cal_metrics[method]['NLL']
    brier = cal_metrics[method]['Brier']
    ece = cal_metrics[method]['ECE']
    print(f"{method:<25s} {nll:>10.4f} {brier:>10.4f} {ece:>10.4f}")

print("\n3. TEMPERATURAS OPTIMIZADAS")
print("-" * 80)
for method in ['baseline', 'mc_dropout', 'decoder_variance']:
    T = temps[method]['T']
    nll_before = temps[method]['nll_before']
    nll_after = temps[method]['nll_after']
    improvement = nll_before - nll_after
    print(f"{method:20s}  T={T:.4f}  NLL: {nll_before:.4f} → {nll_after:.4f} (Δ={improvement:.4f})")

print("\n4. RISK-COVERAGE AUC (menor es mejor)")
print("-" * 80)
for method, auc in auc_summary.items():
    print(f"{method:25s}  AUC={auc:.4f}")

print("\n5. INCERTIDUMBRE: AUROC TP vs FP (mayor es mejor)")
print("-" * 80)
print(f"{'Method':<25s} {'AUROC ↑':>10s} {'Mean Unc TP':>15s} {'Mean Unc FP':>15s} {'Ratio':>10s}")
print("-" * 80)
for method, data in uncertainty_auroc_data.items():
    auroc = data['auroc']
    mean_tp = data['mean_unc_tp']
    mean_fp = data['mean_unc_fp']
    ratio = mean_fp / mean_tp if mean_tp > 0 else 0
    print(f"{method:<25s} {auroc:>10.4f} {mean_tp:>15.6f} {mean_fp:>15.6f} {ratio:>10.2f}x")

print("\n" + "="*80)
print("CONCLUSIONES")
print("="*80)
print("✓ Baseline: rendimiento de referencia sin incertidumbre")
print("✓ Temperature Scaling: mejora calibración sin afectar mAP")
print("✓ MC-Dropout: proporciona incertidumbre epistémica (K pases)")
print("✓ Decoder variance: incertidumbre en single-pass (más eficiente)")
print("✓ Métodos+TS: mejor calibración manteniendo detección")
print("✓ AUROC TP vs FP: valida que incertidumbre detecta errores")
print("="*80)

# Guardar reporte final
final_report = {
    'timestamp': datetime.now().isoformat(),
    'config': CONFIG,
    'detection_metrics': det_metrics,
    'calibration_metrics': cal_metrics,
    'temperatures': temps,
    'risk_coverage_auc': auc_summary,
    'uncertainty_auroc': uncertainty_auroc_data
}

with open(OUTPUT_DIR / 'final_report.json', 'w') as f:
    json.dump(final_report, f, indent=2)

print(f"\nReporte final guardado en: {OUTPUT_DIR / 'final_report.json'}")
print(f"Todos los artefactos en: {OUTPUT_DIR}")


RESUMEN FINAL - COMPARACIÓN DE MÉTODOS

1. MÉTRICAS DE DETECCIÓN (mAP@[0.5:0.95])
--------------------------------------------------------------------------------
baseline                   mAP=0.1705  AP50=0.2785  AP75=0.1705
baseline_ts                mAP=0.1705  AP50=0.2785  AP75=0.1705
mc_dropout                 mAP=0.1823  AP50=0.3023  AP75=0.1811
mc_dropout_ts              mAP=0.1823  AP50=0.3023  AP75=0.1811
decoder_variance           mAP=0.1819  AP50=0.3020  AP75=0.1801
decoder_variance_ts        mAP=0.1819  AP50=0.3020  AP75=0.1801

2. MÉTRICAS DE CALIBRACIÓN
--------------------------------------------------------------------------------
Method                         NLL ↓    Brier ↓      ECE ↓
--------------------------------------------------------------------------------
baseline                      0.7180     0.2618     0.2410
baseline_ts                   0.6930     0.2499     0.1868
mc_dropout                    0.7069     0.2561     0.2034
mc_dropout_ts             

## 15. Visualización Final Comparativa

In [22]:
fig = plt.figure(figsize=(20, 14))
gs = fig.add_gridspec(4, 3, hspace=0.3, wspace=0.3)

# 1. mAP Comparison
ax1 = fig.add_subplot(gs[0, :])
methods = ['baseline', 'baseline_ts', 'mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']
mAPs = [det_metrics[m].get('mAP', 0.0) for m in methods]
colors_map = ['lightblue', 'blue', 'lightcoral', 'red', 'lightgreen', 'green']
bars = ax1.bar(range(len(methods)), mAPs, color=colors_map, alpha=0.7)
ax1.set_xticks(range(len(methods)))
ax1.set_xticklabels([m.replace('_', '\n') for m in methods], fontsize=10)
ax1.set_ylabel('mAP@[0.5:0.95]', fontsize=12)
ax1.set_title('Comparación de mAP entre Métodos', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height, f'{mAPs[i]:.3f}', ha='center', va='bottom', fontsize=9)

# 2. Calibration Metrics Comparison
ax2 = fig.add_subplot(gs[1, 0])
nlls = [cal_metrics[m]['NLL'] for m in methods]
ax2.bar(range(len(methods)), nlls, color=colors_map, alpha=0.7)
ax2.set_xticks(range(len(methods)))
ax2.set_xticklabels([m.replace('_', '\n') for m in methods], fontsize=8)
ax2.set_ylabel('NLL ↓', fontsize=11)
ax2.set_title('Negative Log-Likelihood', fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

ax3 = fig.add_subplot(gs[1, 1])
briers = [cal_metrics[m]['Brier'] for m in methods]
ax3.bar(range(len(methods)), briers, color=colors_map, alpha=0.7)
ax3.set_xticks(range(len(methods)))
ax3.set_xticklabels([m.replace('_', '\n') for m in methods], fontsize=8)
ax3.set_ylabel('Brier Score ↓', fontsize=11)
ax3.set_title('Brier Score', fontsize=12, fontweight='bold')
ax3.grid(axis='y', alpha=0.3)

ax4 = fig.add_subplot(gs[1, 2])
eces = [cal_metrics[m]['ECE'] for m in methods]
ax4.bar(range(len(methods)), eces, color=colors_map, alpha=0.7)
ax4.set_xticks(range(len(methods)))
ax4.set_xticklabels([m.replace('_', '\n') for m in methods], fontsize=8)
ax4.set_ylabel('ECE ↓', fontsize=11)
ax4.set_title('Expected Calibration Error', fontsize=12, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

# 3. Temperature Scaling Effect
ax5 = fig.add_subplot(gs[2, 0])
base_methods = ['baseline', 'mc_dropout', 'decoder_variance']
Ts = [temps[m]['T'] for m in base_methods]
ax5.bar(range(len(base_methods)), Ts, color=['blue', 'red', 'green'], alpha=0.7)
ax5.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='T=1 (sin calibrar)')
ax5.set_xticks(range(len(base_methods)))
ax5.set_xticklabels([m.replace('_', '\n') for m in base_methods], fontsize=10)
ax5.set_ylabel('Temperature T', fontsize=11)
ax5.set_title('Temperaturas Óptimas', fontsize=12, fontweight='bold')
ax5.legend()
ax5.grid(axis='y', alpha=0.3)

# 4. Risk-Coverage AUC
ax6 = fig.add_subplot(gs[2, 1])
unc_methods = ['mc_dropout', 'mc_dropout_ts', 'decoder_variance', 'decoder_variance_ts']
aucs = [auc_summary.get(m, 0.0) for m in unc_methods]
colors_unc = ['lightcoral', 'red', 'lightgreen', 'green']
bars_auc = ax6.bar(range(len(unc_methods)), aucs, color=colors_unc, alpha=0.7)
ax6.set_xticks(range(len(unc_methods)))
ax6.set_xticklabels([m.replace('_', '\n') for m in unc_methods], fontsize=9)
ax6.set_ylabel('AUC (Risk-Coverage) ↓', fontsize=11)
ax6.set_title('Risk-Coverage AUC', fontsize=12, fontweight='bold')
ax6.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars_auc):
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height, f'{aucs[i]:.3f}', ha='center', va='bottom', fontsize=8)

# 5. AUROC TP vs FP (Nueva sección)
ax7 = fig.add_subplot(gs[2, 2])
auroc_methods = list(uncertainty_auroc_data.keys())
aurocs = [uncertainty_auroc_data[m]['auroc'] for m in auroc_methods]
colors_auroc = ['lightcoral', 'red', 'lightgreen', 'green']
bars_auroc = ax7.bar(range(len(auroc_methods)), aurocs, color=colors_auroc, alpha=0.7)
ax7.axhline(y=0.5, color='black', linestyle='--', alpha=0.5, label='Random')
ax7.axhline(y=0.7, color='orange', linestyle='--', alpha=0.5, label='Good threshold')
ax7.set_xticks(range(len(auroc_methods)))
ax7.set_xticklabels([m.replace('_', '\n') for m in auroc_methods], fontsize=9)
ax7.set_ylabel('AUROC (FP detection) ↑', fontsize=11)
ax7.set_title('AUROC: Detección de Errores', fontsize=12, fontweight='bold')
ax7.legend(fontsize=8)
ax7.grid(axis='y', alpha=0.3)
ax7.set_ylim([0, 1])
for i, bar in enumerate(bars_auroc):
    height = bar.get_height()
    ax7.text(bar.get_x() + bar.get_width()/2., height, f'{aurocs[i]:.3f}', ha='center', va='bottom', fontsize=8)

# 6. Resumen de incertidumbre (ratio FP/TP)
ax8 = fig.add_subplot(gs[3, :])
ratios = [uncertainty_auroc_data[m]['mean_unc_fp'] / uncertainty_auroc_data[m]['mean_unc_tp'] 
          if uncertainty_auroc_data[m]['mean_unc_tp'] > 0 else 0 
          for m in auroc_methods]
bars_ratio = ax8.bar(range(len(auroc_methods)), ratios, color=colors_auroc, alpha=0.7)
ax8.axhline(y=1.0, color='black', linestyle='--', alpha=0.5, label='Ratio = 1 (sin diferencia)')
ax8.set_xticks(range(len(auroc_methods)))
ax8.set_xticklabels([m.replace('_', '\n') for m in auroc_methods], fontsize=10)
ax8.set_ylabel('Ratio Mean(Unc_FP) / Mean(Unc_TP)', fontsize=11)
ax8.set_title('Ratio de Incertidumbre: FP vs TP (>1 es deseable)', fontsize=12, fontweight='bold')
ax8.legend()
ax8.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars_ratio):
    height = bar.get_height()
    ax8.text(bar.get_x() + bar.get_width()/2., height, f'{ratios[i]:.2f}x', ha='center', va='bottom', fontsize=9)

plt.suptitle('Fase 5: Comparación Completa de Métodos de Incertidumbre y Calibración', 
             fontsize=16, fontweight='bold', y=0.997)

plt.savefig(OUTPUT_DIR / 'final_comparison_summary.png', dpi=150, bbox_inches='tight')
print(f"\nVisualización final guardada en: {OUTPUT_DIR / 'final_comparison_summary.png'}")
plt.close()

print("\n" + "="*80)
print("FASE 5 COMPLETADA")
print("="*80)
print(f"Todos los resultados guardados en: {OUTPUT_DIR}")
print("\nArchivos generados:")
print("  - config.yaml")
print("  - temperatures.json")
print("  - detection_metrics.json")
print("  - calibration_metrics.json")
print("  - risk_coverage_auc.json")
print("  - uncertainty_auroc.json")
print("  - uncertainty_auroc_comparison.csv")
print("  - final_report.json")
print("  - detection_comparison.csv")
print("  - calibration_comparison.csv")
print("  - reliability_diagrams.png")
print("  - risk_coverage_curves.png")
print("  - uncertainty_analysis.png")
print("  - final_comparison_summary.png")
print("="*80)


Visualización final guardada en: outputs/comparison/final_comparison_summary.png

FASE 5 COMPLETADA
Todos los resultados guardados en: outputs/comparison

Archivos generados:
  - config.yaml
  - temperatures.json
  - detection_metrics.json
  - calibration_metrics.json
  - risk_coverage_auc.json
  - uncertainty_auroc.json
  - uncertainty_auroc_comparison.csv
  - final_report.json
  - detection_comparison.csv
  - calibration_comparison.csv
  - reliability_diagrams.png
  - risk_coverage_curves.png
  - uncertainty_analysis.png
  - final_comparison_summary.png


In [23]:
#!/usr/bin/env python3
"""
Script de verificación de optimizaciones de Fase 5
===================================================

Este script verifica que:
1. Los archivos de fases anteriores existen
2. Los formatos de datos son correctos
3. Las predicciones son compatibles
4. Estima el tiempo que se ahorrará
"""

import json
import sys
from pathlib import Path
from datetime import timedelta


# Colores para terminal
class Colors:
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BLUE = "\033[94m"
    BOLD = "\033[1m"
    END = "\033[0m"


def check_file(path, description):
    """Verifica si un archivo existe y retorna su info"""
    path = Path(path)
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"{Colors.GREEN}✅ {description}{Colors.END}")
        print(f"   Ubicación: {path}")
        print(f"   Tamaño: {size_mb:.2f} MB")
        return True, size_mb
    else:
        print(f"{Colors.RED}❌ {description}{Colors.END}")
        print(f"   {Colors.YELLOW}No encontrado: {path}{Colors.END}")
        return False, 0


def verify_json_format(path, expected_keys):
    """Verifica que un JSON tenga el formato esperado"""
    try:
        with open(path, "r") as f:
            data = json.load(f)

        if not isinstance(data, list) or len(data) == 0:
            return False, "No es una lista o está vacía"

        sample = data[0]
        missing_keys = [k for k in expected_keys if k not in sample]

        if missing_keys:
            return False, f"Faltan keys: {missing_keys}"

        return True, f"{len(data)} registros"
    except Exception as e:
        return False, str(e)


def main():
    print(f"\n{Colors.BOLD}{Colors.BLUE}{'='*70}")
    print("VERIFICACIÓN DE OPTIMIZACIONES - FASE 5")
    print(f"{'='*70}{Colors.END}\n")

    # Paths
    base_dir = Path("..")
    fase2_preds = base_dir / "fase 2" / "outputs" / "baseline" / "preds_raw.json"
    fase3_preds = (
        base_dir / "fase 3" / "outputs" / "mc_dropout" / "preds_mc_aggregated.json"
    )
    fase4_temp = (
        base_dir / "fase 4" / "outputs" / "temperature_scaling" / "temperature.json"
    )

    # Contadores
    files_found = 0
    total_files = 3
    time_saved = 0

    # ========================================================================
    print(f"{Colors.BOLD}1. VERIFICACIÓN DE ARCHIVOS DE FASE 2 (Baseline){Colors.END}")
    print("-" * 70)

    exists, size = check_file(fase2_preds, "Predicciones Baseline")
    if exists:
        files_found += 1
        time_saved += 45  # 45 minutos ahorrados

        # Verificar formato
        valid, info = verify_json_format(
            fase2_preds, ["image_id", "category_id", "bbox", "score"]
        )
        if valid:
            print(f"   {Colors.GREEN}Formato: ✅ Correcto ({info}){Colors.END}")
        else:
            print(f"   {Colors.YELLOW}Formato: ⚠️  {info}{Colors.END}")

    print()

    # ========================================================================
    print(
        f"{Colors.BOLD}2. VERIFICACIÓN DE ARCHIVOS DE FASE 3 (MC-Dropout){Colors.END}"
    )
    print("-" * 70)

    exists, size = check_file(fase3_preds, "Predicciones MC-Dropout")
    if exists:
        files_found += 1
        time_saved += 90  # 90 minutos ahorrados (K=5 es costoso)

        # Verificar formato
        valid, info = verify_json_format(
            fase3_preds, ["image_id", "category_id", "bbox", "score", "uncertainty"]
        )
        if valid:
            print(f"   {Colors.GREEN}Formato: ✅ Correcto ({info}){Colors.END}")
        else:
            print(f"   {Colors.YELLOW}Formato: ⚠️  {info}{Colors.END}")

    print()

    # ========================================================================
    print(
        f"{Colors.BOLD}3. VERIFICACIÓN DE ARCHIVOS DE FASE 4 (Temperature){Colors.END}"
    )
    print("-" * 70)

    exists, size = check_file(fase4_temp, "Temperaturas Optimizadas")
    if exists:
        files_found += 1
        time_saved += 2  # 2 minutos ahorrados

        # Verificar formato
        try:
            with open(fase4_temp, "r") as f:
                temps = json.load(f)

            if "optimal_temperature" in temps:
                T = temps["optimal_temperature"]
                print(f"   {Colors.GREEN}Formato: ✅ Correcto (T={T:.4f}){Colors.END}")
            else:
                print(
                    f"   {Colors.YELLOW}Formato: ⚠️  Falta 'optimal_temperature'{Colors.END}"
                )
        except Exception as e:
            print(f"   {Colors.YELLOW}Formato: ⚠️  Error: {e}{Colors.END}")

    print()

    # ========================================================================
    print(f"{Colors.BOLD}{'='*70}")
    print("RESUMEN")
    print(f"{'='*70}{Colors.END}")

    print(
        f"\n{Colors.BOLD}Archivos encontrados:{Colors.END} {files_found}/{total_files}"
    )

    if files_found == total_files:
        print(f"{Colors.GREEN}✅ TODOS los archivos están disponibles{Colors.END}")
    elif files_found > 0:
        print(f"{Colors.YELLOW}⚠️  Algunos archivos están disponibles{Colors.END}")
    else:
        print(f"{Colors.RED}❌ NO hay archivos disponibles{Colors.END}")

    # Estimación de tiempo
    print(f"\n{Colors.BOLD}Tiempo estimado ahorrado:{Colors.END}")

    if time_saved > 0:
        td = timedelta(minutes=time_saved)
        hours = td.seconds // 3600
        minutes = (td.seconds % 3600) // 60

        print(f"   {Colors.GREEN}⚡ ~{hours}h {minutes}min{Colors.END}")

        if files_found == total_files:
            print(f"\n{Colors.BOLD}Tiempo de ejecución esperado:{Colors.END}")
            print(
                f"   {Colors.GREEN}📊 ~15-20 minutos{Colors.END} (solo Decoder Variance)"
            )
        else:
            missing = total_files - files_found
            est_time = 137 - time_saved  # 137 min total original
            print(f"\n{Colors.BOLD}Tiempo de ejecución esperado:{Colors.END}")
            print(
                f"   {Colors.YELLOW}📊 ~{est_time} minutos{Colors.END} (calcular {missing} método(s) faltante(s))"
            )
    else:
        print(f"   {Colors.RED}❌ 0 minutos{Colors.END}")
        print(f"\n{Colors.BOLD}Tiempo de ejecución esperado:{Colors.END}")
        print(f"   {Colors.RED}📊 ~2 horas{Colors.END} (inferencia completa)")

    # Recomendaciones
    print(f"\n{Colors.BOLD}{'='*70}")
    print("RECOMENDACIONES")
    print(f"{'='*70}{Colors.END}")

    if files_found == total_files:
        print(
            f"{Colors.GREEN}✅ Perfecto! Puedes ejecutar Fase 5 directamente.{Colors.END}"
        )
        print(f"   El notebook usará todos los resultados cacheados.")
    elif files_found == 0:
        print(f"{Colors.YELLOW}⚠️  Ejecuta las siguientes fases primero:{Colors.END}")
        print(f"   1. Fase 2: Genera predicciones baseline")
        print(f"   2. Fase 3: Genera predicciones MC-Dropout")
        print(f"   3. Fase 4: Optimiza temperaturas")
        print(f"\n   O ejecuta Fase 5 directamente (tardará ~2 horas)")
    else:
        print(f"{Colors.YELLOW}⚠️  Tienes optimización parcial.{Colors.END}")

        if not (base_dir / fase2_preds).exists():
            print(f"   • Ejecuta Fase 2 para predicciones baseline")
        if not (base_dir / fase3_preds).exists():
            print(f"   • Ejecuta Fase 3 para predicciones MC-Dropout")
        if not (base_dir / fase4_temp).exists():
            print(f"   • Ejecuta Fase 4 para temperaturas")

        print(f"\n   O ejecuta Fase 5 ahora (ahorrará ~{time_saved} min)")

    print(f"\n{Colors.BOLD}{'='*70}{Colors.END}\n")

    # Exit code
    return 0 if files_found == total_files else 1


if __name__ == "__main__":
    sys.exit(main())



VERIFICACIÓN DE OPTIMIZACIONES - FASE 5

1. VERIFICACIÓN DE ARCHIVOS DE FASE 2 (Baseline)
----------------------------------------------------------------------
✅ Predicciones Baseline
   Ubicación: ../fase 2/outputs/baseline/preds_raw.json
   Tamaño: 3.23 MB
   Formato: ✅ Correcto (22162 registros)

2. VERIFICACIÓN DE ARCHIVOS DE FASE 3 (MC-Dropout)
----------------------------------------------------------------------
✅ Predicciones MC-Dropout
   Ubicación: ../fase 3/outputs/mc_dropout/preds_mc_aggregated.json
   Tamaño: 4.35 MB
   Formato: ⚠️  Faltan keys: ['uncertainty']

3. VERIFICACIÓN DE ARCHIVOS DE FASE 4 (Temperature)
----------------------------------------------------------------------
✅ Temperaturas Optimizadas
   Ubicación: ../fase 4/outputs/temperature_scaling/temperature.json
   Tamaño: 0.00 MB
   Formato: ⚠️  Falta 'optimal_temperature'

RESUMEN

Archivos encontrados: 3/3
✅ TODOS los archivos están disponibles

Tiempo estimado ahorrado:
   ⚡ ~2h 17min

Tiempo de ejecuc

SystemExit: 0